#### 20260109 update to intrinsics summaries from calib.db

In [1]:
import json
import pandas as pd
import glob
import os
import numpy as np

#### set up the infrastructure to read calib.db json output

In [2]:
json_dir = r'C:\CJH\python\FRC\vision\2025\python_2025_multicam_2429\intrinsics'
json_dir = r'C:\CJH\python\FRC\vision\2026\intrinsics'

In [3]:
def get_calibration_df(directory_path):
    """
    Scans a directory for .json calibration files and returns a 
    formatted Pandas DataFrame of intrinsic parameters.
    """
    pattern = os.path.join(directory_path, "**", "*.json")
    json_files = glob.glob(pattern, recursive=True)
    
    all_data = []
    for file in json_files:
        with open(file, 'r') as f:
            data = json.load(f)
            
            # Flatten the camera matrix and resolution into top-level keys
            matrix = data['camera_matrix']
            dist = data['distortion_coefficients']
            
            flat_entry = {
                'camera': data.get('camera'),
                'fx': matrix[0][0],
                'fy': matrix[1][1],
                'cx': matrix[0][2],
                'cy': matrix[1][2],
                'd1': dist[0], 'd2': dist[1], 'd3': dist[2], 'd4': dist[3], 'd5': dist[4],
                'x_res': data['img_size'][0],
                'y_res': data['img_size'][1],
                'avg_reprojection_error': data.get('avg_reprojection_error')
            }
            all_data.append(flat_entry)
            
    return pd.DataFrame(all_data)

In [4]:
# this is not averaged 
df = get_calibration_df(json_dir)
df

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
0,Arducam OV9281 USB Camera (0c45:6366) GREEN,904.250538,904.093847,631.559501,327.181184,0.034515,-0.039206,0.001369,-0.000608,-0.019907,1280,720,0.164207
1,Arducam OV9281 USB Camera (0c45:6366) GREEN,902.551787,902.501630,631.873031,325.488773,0.034539,-0.030149,0.001718,-0.000328,-0.039829,1280,720,0.185005
2,Arducam OV9281 USB Camera (0c45:6366) GREEN,905.845321,905.440379,631.460252,324.066395,0.038168,-0.043184,0.001116,-0.000119,-0.027987,1280,720,0.154939
3,Arducam OV9281 USB Camera (0c45:6366) RED,898.784549,899.151183,630.395824,369.335779,0.041353,-0.057882,0.000098,0.000361,-0.007250,1280,720,0.212901
4,Arducam OV9281 USB Camera (0c45:6366) RED,904.904326,904.807079,626.580344,369.369034,0.045201,-0.055299,0.000799,-0.000983,-0.011861,1280,720,0.139020
5,Arducam OV9281 USB Camera (0c45:6366) RED,903.690298,903.930989,630.313983,369.038240,0.045356,-0.079693,-0.000016,0.000425,0.018643,1280,720,0.168310
6,Arducam OV9281 USB C (0c45:6366) BLUE,907.098629,907.413786,642.437667,353.454982,0.037372,-0.060808,-0.000245,-0.000501,-0.002013,1280,720,0.159565
7,Arducam OV9281 USB C (0c45:6366) BLUE,910.844409,910.503483,642.311810,352.947247,0.034710,-0.052340,-0.000485,-0.000017,-0.007677,1280,720,0.173978
8,Arducam OV9281 USB C (0c45:6366) BLUE,908.147179,907.520047,642.686962,350.731126,0.027600,-0.038416,-0.001653,-0.000187,-0.018261,1280,720,0.174499
9,Arducam OV9281 USB C (0c45:6366) YELLOW,911.167954,910.174914,638.981835,430.251980,0.031861,-0.051294,-0.001721,-0.002335,-0.007824,1280,720,0.171557


---
####  now average the camera data in case there are outliers

In [5]:
def get_model_averages(df):
    """Returns the mean intrinsic values grouped by camera model."""
    return df.groupby("camera").mean(numeric_only=True).reset_index()

In [6]:
averages = get_model_averages(df)
averages

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
0,Arducam OV9281 USB C (0c45:6366) BLUE,908.696739,908.479105,642.478813,352.377785,0.033227,-0.050521,-0.000794,-0.000235,-0.009317,1280.0,720.0,0.169348
1,Arducam OV9281 USB C (0c45:6366) YELLOW,912.011490,911.562573,641.405695,431.376325,0.037267,-0.061672,-0.000816,-0.001104,0.002426,1280.0,720.0,0.181807
2,Arducam OV9281 USB Camera (0c45:6366) GREEN,904.215882,904.011952,631.630928,325.578784,0.035741,-0.037513,0.001401,-0.000351,-0.029241,1280.0,720.0,0.168050
3,Arducam OV9281 USB Camera (0c45:6366) RED,902.459724,902.629750,629.096717,369.247685,0.043970,-0.064291,0.000294,-0.000066,-0.000156,1280.0,720.0,0.173410


In [7]:
# Filter for any camera string containing the C920 hardware ID
c920_all = df[df['camera'].str.contains('046d:08e5', case=False)]
c920_all

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error


In [8]:
for _, row in averages.iterrows():
    # Construct the dictionary for unscaled intrinsics
    intrinsics_dict = {
        "fx": round(row['fx'], 2),
        "fy": round(row['fy'], 2),
        "cx": round(row['cx'], 2),
        "cy": round(row['cy'], 2)
    }
    
    # Extract unscaled distortion coefficients into a list
    dist_list = [
        round(row['d1'], 8), 
        round(row['d2'], 8), 
        round(row['d3'], 8), 
        round(row['d4'], 8), 
        round(row['d5'], 8)
    ]
    
    # Print with the camera name preamble and distortions on the next line
    print(f"{row['camera']} :")
    print(f"    \"intrinsics\": {json.dumps(intrinsics_dict)},")
    print(f"    \"distortions\": {dist_list},")

Arducam OV9281 USB C (0c45:6366) BLUE :
    "intrinsics": {"fx": 908.7, "fy": 908.48, "cx": 642.48, "cy": 352.38},
    "distortions": [0.0332274, -0.05052129, -0.00079437, -0.00023466, -0.00931715],
Arducam OV9281 USB C (0c45:6366) YELLOW :
    "intrinsics": {"fx": 912.01, "fy": 911.56, "cx": 641.41, "cy": 431.38},
    "distortions": [0.03726703, -0.06167168, -0.00081609, -0.00110447, 0.00242587],
Arducam OV9281 USB Camera (0c45:6366) GREEN :
    "intrinsics": {"fx": 904.22, "fy": 904.01, "cx": 631.63, "cy": 325.58},
    "distortions": [0.03574071, -0.03751287, 0.00140087, -0.00035136, -0.02924075],
Arducam OV9281 USB Camera (0c45:6366) RED :
    "intrinsics": {"fx": 902.46, "fy": 902.63, "cx": 629.1, "cy": 369.25},
    "distortions": [0.04397004, -0.06429124, 0.00029384, -6.56e-05, -0.00015588],


---
#### tools to scale down if we choose to

In [9]:
def scale_intrinsics(target_res, source_data, use_max_focal=True):
    """
    Scales intrinsic parameters from a source resolution to a target resolution.
    
    Args:
        target_res (tuple): (width, height) e.g., (640, 360)
        source_data (dict/Series): Dictionary containing fx, fy, cx, cy, x_res, y_res
        use_max_focal (bool): If True, uses the larger scale factor for focal length 
                              (standard for maintaining FOV during cropping).
    """
    target_x, target_y = target_res
    source_x, source_y = source_data['x_res'], source_data['y_res']
    
    scale_x = target_x / source_x
    scale_y = target_y / source_y
    
    # Focal length scaling
    f_scale = max(scale_x, scale_y) if use_max_focal else (scale_x, scale_y)
    
    if isinstance(f_scale, tuple):
        fx = source_data['fx'] * f_scale[0]
        fy = source_data['fy'] * f_scale[1]
    else:
        fx = source_data['fx'] * f_scale
        fy = source_data['fy'] * f_scale

    # Principal point scaling
    cx = source_data['cx'] * scale_x
    cy = source_data['cy'] * scale_y
    
    # Calculate FOV for verification
    fov_h = 2 * np.degrees(np.arctan(target_x / (2 * fx)))
    fov_v = 2 * np.degrees(np.arctan(target_y / (2 * fy)))
    
    return {
        'fx': round(fx, 3), 'fy': round(fy, 3),
        'cx': round(cx, 2), 'cy': round(cy, 2),
        'fov_h': round(fov_h, 1), 'fov_v': round(fov_v, 1)
    }

In [10]:
# target resolution for the scaled output
target_resolution = (640, 360)

for _, row in averages.iterrows():
    # Scale the average data for this camera
    scaled = scale_intrinsics(target_resolution, row)
    
    # Format the intrinsics dictionary
    intrinsics_dict = {
        "fx": scaled['fx'],
        "fy": scaled['fy'],
        "cx": scaled['cx'],
        "cy": scaled['cy']
    }
    
    # Extract distortion coefficients into a list
    dist_list = [
        round(row['d1'], 8), 
        round(row['d2'], 8), 
        round(row['d3'], 8), 
        round(row['d4'], 8), 
        round(row['d5'], 8)
    ]
    
    # Print with the camera name preamble and distortions on the next line
    print(f"{row['camera']} (640x360) :")
    print(f"    \"intrinsics\": {json.dumps(intrinsics_dict)},")
    print(f"    \"distortions\": {dist_list},")

Arducam OV9281 USB C (0c45:6366) BLUE (640x360) :
    "intrinsics": {"fx": 454.348, "fy": 454.24, "cx": 321.24, "cy": 176.19},
    "distortions": [0.0332274, -0.05052129, -0.00079437, -0.00023466, -0.00931715],
Arducam OV9281 USB C (0c45:6366) YELLOW (640x360) :
    "intrinsics": {"fx": 456.006, "fy": 455.781, "cx": 320.7, "cy": 215.69},
    "distortions": [0.03726703, -0.06167168, -0.00081609, -0.00110447, 0.00242587],
Arducam OV9281 USB Camera (0c45:6366) GREEN (640x360) :
    "intrinsics": {"fx": 452.108, "fy": 452.006, "cx": 315.82, "cy": 162.79},
    "distortions": [0.03574071, -0.03751287, 0.00140087, -0.00035136, -0.02924075],
Arducam OV9281 USB Camera (0c45:6366) RED (640x360) :
    "intrinsics": {"fx": 451.23, "fy": 451.315, "cx": 314.55, "cy": 184.62},
    "distortions": [0.04397004, -0.06429124, 0.00029384, -6.56e-05, -0.00015588],


---
#### show min/max on each parameter

In [11]:
# Assuming 'df' is the DataFrame returned by get_calibration_df()
intrinsics_cols = ['fx', 'fy', 'cx', 'cy', 'avg_reprojection_error']

# Group by the camera model and calculate min, max, and mean for each parameter
summary_table = df.groupby('camera')[intrinsics_cols].agg(['min', 'max', 'mean'])

# Optional: Clean up the column names for better readability
summary_table.columns = ['_'.join(col).strip() for col in summary_table.columns.values]

# Display the result
summary_table.reset_index()

,camera,fx_min,fx_max,fx_mean,fy_min,fy_max,fy_mean,cx_min,cx_max,cx_mean,cy_min,cy_max,cy_mean,avg_reprojection_error_min,avg_reprojection_error_max,avg_reprojection_error_mean
0,Arducam OV9281 USB C (0c45:6366) BLUE,907.098629,910.844409,908.696739,907.413786,910.503483,908.479105,642.311810,642.686962,642.478813,350.731126,353.454982,352.377785,0.159565,0.174499,0.169348
1,Arducam OV9281 USB C (0c45:6366) YELLOW,911.167954,912.570942,912.011490,910.174914,912.593903,911.562573,638.981835,643.227822,641.405695,430.251980,432.685746,431.376325,0.171557,0.189816,0.181807
2,Arducam OV9281 USB Camera (0c45:6366) GREEN,902.551787,905.845321,904.215882,902.501630,905.440379,904.011952,631.460252,631.873031,631.630928,324.066395,327.181184,325.578784,0.154939,0.185005,0.168050
3,Arducam OV9281 USB Camera (0c45:6366) RED,898.784549,904.904326,902.459724,899.151183,904.807079,902.629750,626.580344,630.395824,629.096717,369.038240,369.369034,369.247685,0.139020,0.212901,0.173410
